In [1]:
# Panggil library yang dibutuhkan
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
# Digunakan untuk menghapus dataframe jika ada kesalahan
del df

In [ ]:
# Membuat dataframe
df = pd.DataFrame()

In [ ]:
# List untuk menyimpan data yang diambil
listProduct = []
listType = []
listPrice = []
listRating = []
listReview = []

In [ ]:
# Link yang akan di-scrape
url = 'https://www.uniqlo.com/us/en/men/tops'

In [ ]:
driver = webdriver.Chrome()
driver.get(url)
time.sleep(20) # Tunggu 20 detik agar halaman ter-load sembari di-scroll

soup = BeautifulSoup(driver.page_source, 'html.parser')

# Mengambil semua produk yang ada di halaman
products = soup.find_all('a', {'class': 'link ito-padding-horizontal-0 ito-padding-vertical-0 ec-link product-tile__link'})

for product in products:
    # Produk
    produk = product.find('div', {
        'class': 'product-tile--vertical__content-area',
        'class': 'typography typography-reset ito-font-family-uq-en-like ito-font-weight-400 ito-font-size-15 ito-font-lh-1-3 text-align-left text-transform-normal ito-margin-bottom-8'
        })
    listProduct.append(produk.get_text().strip() if produk else None) # Menggunakan strip() untuk menghapus spasi di awal dan akhir

    # Tipe
    tipe = product.find('div', {
        'class': 'product-tile--vertical__content-area',
        'class': 'typography typography-reset ito-font-family-uq-en-like ito-font-weight-400 ito-font-size-13 ito-font-lh-1-2 ito-secondary-text-color text-align-left text-transform-normal ito-margin-bottom-4'
    })
    listType.append(tipe.get_text().strip() if tipe else None)

    # Harga
    price = product.find('div', {
        'data-testid': 'ITOTypography',
        'class': 'typography typography-reset ito-font-family-uq-brand-specific ito-font-weight-400 ito-font-size-20 ito-font-lh-1-2 ito-attention-text-color ito-font-ls-0-36 text-align-left text-transform-normal ito-margin-right-12'
    })
    if not price:
        price = product.find('div', {
            'data-testid': 'ITOTypography',
            'class': 'typography typography-reset ito-font-family-uq-brand-specific ito-font-weight-400 ito-font-size-20 ito-font-lh-1-2 ito-primary-text-color ito-font-ls-0-36 text-align-left text-transform-normal ito-margin-right-12'
        })
    listPrice.append(price.get_text().strip() if price else None)

    # Rating
    rating = product.find('div', {'class': 'fr-ec-rating-average-product-tile fr-ec-rating-static__average-value'})
    listRating.append(rating.get_text().strip() if rating else None)

    # Review
    review = product.find('div', {'class': 'fr-ec-rating-static__count-product-tile fr-ec-caption--extra-small'})
    listReview.append(review.get_text().strip() if review else None)

driver.quit() # Memberhentikan driver setelah selesai

# Menambahkan data pada list ke dalam dataframe
df['Produk'] = listProduct 
df['Tipe'] = listType
df['Harga'] = listPrice
df['Rating'] = listRating
df['Review'] = listReview

display(df) # Menampilkan dataframe

KeyboardInterrupt: 

In [ ]:
df.to_csv('clothes.csv',index=False) # Menyimpan dataframe ke file CSV

In [ ]:
# Membaca file clothes.csv 
df = pd.read_csv('clothes.csv') 
df.head()

,Produk,Tipe,Harga,Rating,Review
0,SUPIMA® Cotton T-Shirt,"UNISEX, XXS-3XL",$19.90,4.6,(899)
1,SUPIMA® Cotton T-Shirt,"UNISEX, XXS-XS",$9.90,4.6,(899)
2,AIRism Cotton Oversized T-Shirt | Half-Sleeve,"UNISEX, XXS-3XL",$19.90,4.8,(999+)
3,AIRism Cotton T-Shirt | Long Sleeve,"UNISEX, XXS-3XL",$24.90,4.8,(428)
4,AIRism Cotton T-Shirt | Long Sleeve,"UNISEX, XL-3XL",$14.90,4.8,(428)


In [ ]:
df.columns # Menampilkan nama kolom pada dataframe

Index(['Produk', 'Tipe', 'Harga', 'Rating', 'Review'], dtype='object')

In [ ]:
df.isnull().sum() # Menampilkan jumlah nilai null pada setiap kolom

Produk     0
Tipe       0
Harga      0
Rating    29
Review    29
dtype: int64

In [ ]:
df.info() # Menampilkan info dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Produk  144 non-null    object 
 1   Tipe    144 non-null    object 
 2   Harga   144 non-null    object 
 3   Rating  115 non-null    float64
 4   Review  115 non-null    object 
dtypes: float64(1), object(4)
memory usage: 5.8+ KB


In [ ]:
# Menghapus '$' dari kolom Harga dan mengubah tipe data menjadi float
df['Harga'] = df.Harga.str.replace('$', '')
df['Harga'] = df.Harga.astype(float)

In [ ]:
# Menghapus karakter yang tidak diinginkan dari kolom Review dan mengubah tipe data menjadi float
df['Review'] = df.Review.str.replace('(', '')
df['Review'] = df.Review.str.replace(')', '')
df['Review'] = df.Review.str.replace('+', '')
df['Review'] = df.Review.astype(float)

In [ ]:
# Mengambil kata pertama yang dipisahkan oleh koma pada kolom Tipe
# Mengubah tipe data kolom Tipe dan Produk menjadi string
df['Tipe'] = df.Tipe.str.split(',').str[0].str.strip()
df['Tipe'] = df.Tipe.astype('string')
df['Produk'] = df.Produk.astype('string')

In [ ]:
df # Cek kembali dataframe setelah pembersihan data

,Produk,Tipe,Harga,Rating,Review
0,SUPIMA® Cotton T-Shirt,UNISEX,19.9,4.6,899.0
1,SUPIMA® Cotton T-Shirt,UNISEX,9.9,4.6,899.0
2,AIRism Cotton Oversized T-Shirt | Half-Sleeve,UNISEX,19.9,4.8,999.0
3,AIRism Cotton T-Shirt | Long Sleeve,UNISEX,24.9,4.8,428.0
4,AIRism Cotton T-Shirt | Long Sleeve,UNISEX,14.9,4.8,428.0
...,...,...,...,...,...
139,PEANUTS UT Graphic T-Shirt,UNISEX,24.9,5.0,4.0
140,PEACE FOR ALL Graphic T-Shirt | MOOMIN,UNISEX,24.9,4.8,29.0
141,PEACE FOR ALL Graphic T-Shirt | Hakuju Kuiseko,UNISEX,24.9,4.9,22.0
142,PEACE FOR ALL Graphic T-Shirt | Takeshi Yoro,UNISEX,24.9,4.8,16.0


In [ ]:
df.info() # Cek kembali info dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Produk  144 non-null    string 
 1   Tipe    144 non-null    string 
 2   Harga   144 non-null    float64
 3   Rating  115 non-null    float64
 4   Review  115 non-null    float64
dtypes: float64(3), string(2)
memory usage: 5.8 KB


In [ ]:
df.describe().style.format("{:.2f}") # Cek deskripsi statistik dari dataframe

,Harga,Rating,Review
count,144.000000,115.000000,115.000000
mean,24.240278,4.728696,80.486957
std,8.865872,0.535046,187.282850
min,5.900000,1.000000,1.000000
25%,19.900000,4.700000,4.000000
50%,24.900000,4.800000,12.000000
75%,24.900000,5.000000,53.000000
max,49.900000,5.000000,999.000000


In [111]:
df.to_csv('clothes-clean.csv',index=False) # Menyimpan dataframe final ke file csv baru

In [ ]:
# Membaca file clothes-clean.csv
df = pd.read_csv('clothes-clean.csv') 
df.head()

,Produk,Tipe,Harga,Rating,Review
0,SUPIMA® Cotton T-Shirt,UNISEX,19.9,4.6,899.0
1,SUPIMA® Cotton T-Shirt,UNISEX,9.9,4.6,899.0
2,AIRism Cotton Oversized T-Shirt | Half-Sleeve,UNISEX,19.9,4.8,999.0
3,AIRism Cotton T-Shirt | Long Sleeve,UNISEX,24.9,4.8,428.0
4,AIRism Cotton T-Shirt | Long Sleeve,UNISEX,14.9,4.8,428.0
